<a href="https://colab.research.google.com/github/AAKAAASSHHH24/NLP-BASICS/blob/main/text_generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import os
import time
import tensorflow as tf


In [2]:
pwd

'/content'

In [3]:
os.makedirs("data", exist_ok = True)
os.chdir ('data')

In [4]:
!curl https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt > "shakespeare.txt"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  11.6M      0 --:--:-- --:--:-- --:--:-- 11.8M


In [5]:
#this can be done using config.yaml
class Config:
  path_to_file = os.path.join("/content/data","shakespeare.txt")
  seq_length = 100  # AT ONE GO HOW MUCH IS THE LENGTH OF THE WORDS

  batch_size = 64
  buffer_size = 10000   # KEEP THIS MUCH DATA IN BUFFER OR RAM (pick randomly from this buffer FOR FASTER READ/WRITE DURING TRAINING)

  embedding_dim =256

  rnn_units = 1024

  epochs = 30
  checkpoint_dir = "./training_ckpt"


In [6]:
Config.path_to_file

'/content/data/shakespeare.txt'

In [7]:
text = open(Config.path_to_file, "rb").read().decode(encoding = 'utf-8')
text[:100]

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'

In [8]:
#letters of the text
sorted(set(text))

['\n',
 ' ',
 '!',
 '$',
 '&',
 "'",
 ',',
 '-',
 '.',
 '3',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'X',
 'Y',
 'Z',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [9]:
characters =sorted(set(text))
len(characters)

65

In [39]:
idx2char = np.array(characters)
idx2char

array(['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?',
       'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
       'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z',
       'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
       'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z'],
      dtype='<U1')

In [10]:
#vectorization
char_dict = {char:idx for idx,char in enumerate(characters)}
char_dict

{'\n': 0,
 ' ': 1,
 '!': 2,
 '$': 3,
 '&': 4,
 "'": 5,
 ',': 6,
 '-': 7,
 '.': 8,
 '3': 9,
 ':': 10,
 ';': 11,
 '?': 12,
 'A': 13,
 'B': 14,
 'C': 15,
 'D': 16,
 'E': 17,
 'F': 18,
 'G': 19,
 'H': 20,
 'I': 21,
 'J': 22,
 'K': 23,
 'L': 24,
 'M': 25,
 'N': 26,
 'O': 27,
 'P': 28,
 'Q': 29,
 'R': 30,
 'S': 31,
 'T': 32,
 'U': 33,
 'V': 34,
 'W': 35,
 'X': 36,
 'Y': 37,
 'Z': 38,
 'a': 39,
 'b': 40,
 'c': 41,
 'd': 42,
 'e': 43,
 'f': 44,
 'g': 45,
 'h': 46,
 'i': 47,
 'j': 48,
 'k': 49,
 'l': 50,
 'm': 51,
 'n': 52,
 'o': 53,
 'p': 54,
 'q': 55,
 'r': 56,
 's': 57,
 't': 58,
 'u': 59,
 'v': 60,
 'w': 61,
 'x': 62,
 'y': 63,
 'z': 64}

In [11]:
text_as_int = np.array(list(char_dict.values()))
text_as_int

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64])

In [12]:
examples_per_epoch = len(text)//(Config.seq_length + 1)
examples_per_epoch


11043

In [16]:
print("hi\nthere"), print(repr('hi\nthere'))  #demonstrating use of repr()

hi
there
'hi\nthere'


(None, None)

In [13]:
idx_2_char = {idx:char for idx,char in enumerate(characters)}
idx_2_char

{0: '\n',
 1: ' ',
 2: '!',
 3: '$',
 4: '&',
 5: "'",
 6: ',',
 7: '-',
 8: '.',
 9: '3',
 10: ':',
 11: ';',
 12: '?',
 13: 'A',
 14: 'B',
 15: 'C',
 16: 'D',
 17: 'E',
 18: 'F',
 19: 'G',
 20: 'H',
 21: 'I',
 22: 'J',
 23: 'K',
 24: 'L',
 25: 'M',
 26: 'N',
 27: 'O',
 28: 'P',
 29: 'Q',
 30: 'R',
 31: 'S',
 32: 'T',
 33: 'U',
 34: 'V',
 35: 'W',
 36: 'X',
 37: 'Y',
 38: 'Z',
 39: 'a',
 40: 'b',
 41: 'c',
 42: 'd',
 43: 'e',
 44: 'f',
 45: 'g',
 46: 'h',
 47: 'i',
 48: 'j',
 49: 'k',
 50: 'l',
 51: 'm',
 52: 'n',
 53: 'o',
 54: 'p',
 55: 'q',
 56: 'r',
 57: 's',
 58: 't',
 59: 'u',
 60: 'v',
 61: 'w',
 62: 'x',
 63: 'y',
 64: 'z'}

In [32]:
idx_2_char[2]

'!'

In [20]:
char_value = list(char_dict.values())
char_value

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64]

In [17]:
text[:13]

'First Citizen'

In [41]:
# creating a dataset from the above
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) # API supports writing descriptive and efficient input pipelines.

for i in char_dataset.take(20):
  print(idx2char[i.numpy()])

#print(char_dataset)



 
!
$
&
'
,
-
.
3
:
;
?
A
B
C
D
E
F
G


In [43]:
sequences = char_dataset.batch(Config.seq_length + 1, drop_remainder = True)  # creating batches of 100 and handling the remainder, passing an array as dataset
#print(idx_2_char.values())
for item in sequences.take(20):  # taking this many number of batches
  to_print = repr("".join([idx2char[c] for c in item.numpy()]))
  print(to_print)
  print(len(to_print))

In [ ]:
def word_to_number(text):
  numberfied = np.array(list(char_dict[char] for i, char in enumerate (text)))
  return numberfied

 

In [ ]:
word_to_number(text[:13])

array([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52])

In [ ]:
list(char_dict.keys())[:13]

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?']

In [ ]:
def word_extractor(text):
  for i in range(len(text)):
    word = [text]
